### Collecting Data

The bunch of code below exists solely to set up a number of the things we will be using often. The first few lines import some code that is part of the Python3 library:

- `urlopen` will help us pull data from websites.
- `BytesIO` will help us "convert" in-memory data, and treat it like real files.
- `ZipFile` will help us work with zip archives
- `sys` provides a value (`maxsize`) which we'll use for an impossibly large narge.


- `pprint` really is only here for development. It prints data in an easier to read manner.

After that we define a function, `url_to_list()` which will open a URL of a ZIP archive, and will return it as a character delimitted list.

In [1]:
from urllib.request import urlopen
from io import BytesIO, StringIO
from zipfile import ZipFile

import sys

from pprint import pprint

def url_to_list(url: str, file: str) -> list:
    with urlopen(url) as data:
        archive = ZipFile(BytesIO(data.read()))
        
    text = archive.open(file).read().decode()
    text = text.replace(u'"', u'')
    
    return [line.split(u',') for line in text.split(u'\n')]

Here we're doing a few different things.

1. We're going to store the URLs in a variable. Really this is just for convenience...
  1. The OGOR-A links are actually stored in a list of tuples. This is because there is a different url for every year's files. (See the Appendix section at the end.) Storing it in this tuple will actually make it easier to get all of the data. For the purposes of recreating the Delta House file you gave me, I'll actually specify the year 2015. (Really that's mostly just because that's the first column on the spreadsheet.)
  
2. Then we'll open (download) the BSEE files at each URL, and store them in their own variable.
  1. Within each variable, we'll also get rid of some of the "junk data". Empty rows, header rows, etc.
    1. That's accomplished with the list slicing function at the end of each line. `[:-1]` slices out everything but the last element.
  
3. Finally, we're going to define a function that will convert the information from the `deep_water_field_leases.txt` file into a dictionary (remember, `key-value pairs`) with the Block Num as the Key, and the Field Name as the Value.
  1. We also use that `.replace()` method because the Block Num information includes some whitespace in between the letters and numbers.
    1. In other words, because the Block Num information as stored in this file has a bunch of whitespace in it (`WC    41`), which is of a varying length (and different to that of the `ogoradelimit.txt` file), we just go ahead and get rid of the whitespace entirely.

In [139]:
ogora_links = [
    ('https://www.data.bsee.gov/Production/Files/ogora1996delimit.zip', u'ogora1996delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora1997delimit.zip', u'ogora1997delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora1998delimit.zip', u'ogora1998delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora1999delimit.zip', u'ogora1999delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2000delimit.zip', u'ogora2000delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2001delimit.zip', u'ogora2001delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2002delimit.zip', u'ogora2002delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2003delimit.zip', u'ogora2003delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2004delimit.zip', u'ogora2004delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2005delimit.zip', u'ogora2005delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2006delimit.zip', u'ogora2006delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2007delimit.zip', u'ogora2007delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2008delimit.zip', u'ogora2008delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2009delimit.zip', u'ogora2009delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2010delimit.zip', u'ogora2010delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2011delimit.zip', u'ogora2011delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2012delimit.zip', u'ogora2012delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2013delimit.zip', u'ogora2013delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2014delimit.zip', u'ogora2014delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2015delimit.zip', u'ogora2015delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2016delimit.zip', u'ogora2016delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2017delimit.zip', u'ogora2017delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogora2018delimit.zip', u'ogora2018delimit.txt'),
    ('https://www.data.bsee.gov/Production/Files/ogoradelimit.zip', u'ogoradelimit.txt')]

ogora_data = url_to_list(url = ogora_links[19][0], file = ogora_links[19][1])[:-1]
ogora_data = ogora_data + url_to_list(url = ogora_links[20][0], file = ogora_links[20][1])[:-1]
ogora_data = ogora_data + url_to_list(url = ogora_links[21][0], file = ogora_links[21][1])[:-1]

fields_link = u'https://www.data.bsee.gov/Other/Files/DeepQualRawData.zip'

fields_data = url_to_list(url = fields_link, file = u'DeepQualRawData/mv_deep_water_field_leases.txt')[1:-1]

def create_field_name_dictionary(fields: list) -> dict:    
    dictionary = dict()

    for row in fields:
        blocknum = row[2] + row[3]
        blocknum = blocknum.replace(u' ', u'')

        dictionary[blocknum] = row[0]
    
    return dictionary

fields_dict = create_field_name_dictionary(fields = fields_data)

### Data Manipulation
#### Set Up and Frameworking

Now we'll get started manupulating all of the data that we have, to format it into data-structures that we can work with. We're going to do the first of a two-step process, where we create a data structure (in this case a dictionary), with the `key` being the WellZone of the row.

Reminder that dictionaries have a requirement that there by a unique identifier as the `key` in the `key-value` pair. That's why we use the WellZone value, instead of something else.

We need a unique value for the keys. However, the `ogoradelimit.txt` file will actually have a 'data point' for each WellZone for (up to) each month in the year's data. That doesn't sound very unique!

Fortunately, the first time we go through this two-step process, we're only setting up the framework for our data, and we don't care about the data points. If we wanted to be careful, we could put in an `if` statement to ensure we only record that WellZone, if it's not already in the dictionary.

Like I said, it doesn't matter too much, because the first time through, we don't care about the actual datapoints, just the WellZone details. For completeness, though, see the commented out lines 22-32 below.

In [144]:
ogora_dict = {}

for row in ogora_data:
    API = row[8]
    Int = row[15]
    BlockNum = row[10]
   
  
    WellZone = API + Int
    
    ogora_dict[WellZone] = {
        u'BlockNum': BlockNum,
        u'API': API,
        u'Int': Int,
        u'DailyOil': dict(),
        u'DailyGas': dict(),
        u'DailyWater': dict(),
        u'ProductionDays': dict()
    }
    
#     if WellZone not in ogora_dict.keys():
#         ogora_dict[WellZone] = {
#             u'BlockNum': BlockNum,
#             u'API': API,
#             u'Int': Int,
#             u'FieldName': u'MISSING_DATA',
#             u'DailyOil': dict(),
#             u'DailyGas': dict(),
#             u'DailyWater': dict(),
#             u'ProductionDays': dict()
#         }

Next, we're going to iterate through each `WellZone`s in the `ogora_dict` variable, and assign a Field Name to it, provided one exists. The `try-except` code structure allows us to try and do something, but if a specific exception occurs, do something differently.

A `KeyError` is an exception that occurs when you reference a `key` in a dictionary that does not exist. In the case of the `fields_dict` dictionary, if a `key` does not exist, it means we couldn't find the Field Name in the `mv_deep_water_field_leases.txt` file. To ensure we still put in enough data, we'll fill that WellZone with a `u'NAME_NOT_FOUND'`.

In [145]:
for well in list(ogora_dict.keys()):
    a = ogora_dict[well][u'BlockNum'].replace(u' ', u'')
    
    try:
        ogora_dict[well][u'FieldName'] = fields_dict[a]
    
    except KeyError:
        ogora_dict[well][u'FieldName'] = u'NAME_NOT_FOUND'

At this point we've set up the Framework for how we want to sort our data. In fact, if we look at a random WellZone, we can actually see the framework. (I did some looking around later for a WellZone with some good information, so we're going to use that as our sample throughout.)

In [146]:
sample_wz = u'608054001101S01'
pprint(ogora_dict[sample_wz])

{'API': '608054001101',
 'BlockNum': 'AC   25',
 'DailyGas': {},
 'DailyOil': {},
 'DailyWater': {},
 'FieldName': 'Hoover',
 'Int': 'S01',
 'ProductionDays': {}}


So now, we're going to start inputting the data into this framework.

#### Data Entry
In this *second* run through of the `ogora_data` erm.... data, we're actually going to add the oil, gas, water, and production days information into the `ogora_dict` data structure.

We use the same approach to get the `WellZone` variable, combining the `API` and `Int` values. And then within each of the appropriate `u'DailyOil'`, `u'DailyGas'`, `u'DailyWater'`, and `u'ProductionDays'` dictionaries, add the data for each month.

This isn't really written all that overly clearly... But if you compare the code below, with the code above, you'll see that the things like `row[8]` is just your `API`, `row[15]` is the `Int`, etc.

You can compare with the spec information on the BSEE site, or by looking at the data in the `ogoradelimit.txt` files, to confirm that `row[5]` is the Daily Oil number, `row[6]` is Daily Gas, `row[7]` is Daily Water, and finally `row[3]` is the days of Production in that month.

In [148]:
for row in ogora_data:
    ogora_dict[row[8] + row[15]][u'DailyOil'][row[2]] = row[5]
    ogora_dict[row[8] + row[15]][u'DailyGas'][row[2]] = row[6]
    ogora_dict[row[8] + row[15]][u'DailyWater'][row[2]] = row[7]
    ogora_dict[row[8] + row[15]][u'ProductionDays'][row[2]] = row[3]

Let's take a look at that sample WellZone again, now that we've added data to it.

In [149]:
pprint(ogora_dict[sample_wz])

{'API': '608054001101',
 'BlockNum': 'AC   25',
 'DailyGas': {'201501': '000039611',
              '201502': '000031359',
              '201503': '000035747',
              '201504': '000031226',
              '201505': '000028646',
              '201506': '000024157',
              '201507': '000037648',
              '201508': '000021362',
              '201509': '000015559',
              '201510': '000027842',
              '201511': '000028931',
              '201512': '000026885',
              '201601': '000021715',
              '201602': '000026251',
              '201603': '000022491',
              '201604': '000010367',
              '201605': '000025213',
              '201606': '000019664',
              '201607': '000020054',
              '201608': '000019580',
              '201609': '000019719',
              '201610': '000019231',
              '201611': '000018248',
              '201612': '000016021',
              '201701': '000019858',
              '201702': '00

It has a lot more information on it now! Let's take this information, and make it into the format of your Excel Spreadsheet.

### Data Formatting
In order to do that, we're going to have to do some more data manipulation. Let's start by defining a couple helper functions.

- The first one is going to help us to get the minimum, and maximum 'months', and the total number of months with data.
  - Depending on your source data, this could be something like `201501`, `201512`, and `12`; or it could be far larger like `199601`, `201812`, `276`.
  - To calculate the minimums and maximums, we set the first dates to be 'impossibly' large and small, respectively, to allow for us to replace values with smaller or larger ones, also respectively. Fortunately, Python gives those values with the maximum integer size it can handle, and `0`.
    - Within these calculations, we also use two kinda cool 'functions':
      - The first, `[variable] // 100` does integer division. In other words, it ignores the values after the decimal point. (`1234 // 10 = 123`) In effect, we're just truncating the last two digits, to get the year.
      - The second, `[variable] % 100` is the `modulo` operator. This performs a division operation, and returns *just* the remainder. (`1234 % 10 = 4`)
    - These are done to basically separate out the year and the month from the data given in the BSEE files.
- Second, we're going to define what's called a `generator` function, to produce each of the months, between these first and last dates. Unfortunately, we can't just use the built-in `range()` function, because there's no way to have it 'know' to skip from 201512 to 201601. By default, it would go to 201513, which is obviously not a real month!
  - The generator will run until that `while` loop is no longer true, and each time it runs through, it will `yield` whatever value we ask it to; in this case, a `str(value)`.
  - The other bits should hopefully make sense. We're checking to see if the final two digits are `12`, and if they are, rather than simply incrementing by `1`, we're going to add `100` (in effect incrementing the *year*, and then subtract `11`, resetting the *month* to `1`. Otherwise, we simply increment by `1`.
  
- Finally, by convention, I'm assigning these first, last and total month values to variables. By convention, a 'global' variable is named in ALL_CAPS. But I'm mostly trying to draw attention to them, because I will be referring to them again later.
  - I've also put in some print statements just to show what the output of these functions will look like.

In [150]:
def month_range(data: dict) -> tuple:
    first_month = sys.maxsize
    last_month = 0
    
    for wz in data:
        min_month = int(min(data[wz][u'ProductionDays'].keys()))
        max_month = int(max(data[wz][u'ProductionDays'].keys()))

        if min_month < first_month:
            first_month = min_month

        if max_month > last_month:
            last_month = max_month
            
    months = ((last_month // 100) - (first_month // 100)) * 12 + (last_month % 100 - first_month % 100) + 1
    return first_month, last_month, months

def date_generator(start: int, stop: int) -> str:
    value = start
    while value <= stop:
        yield str(value)
        
        if value % 100 == 12:
            value += 100
            value -= 11

        else:
            value += 1

            FIRST_MONTH, LAST_MONTH, TOTAL_MONTHS = month_range(ogora_dict)

print(f'The month_range() function gets us that the first month is {FIRST_MONTH},'
      f'the last month is {LAST_MONTH}, and the total number of months is {TOTAL_MONTHS}')
print(f'Also, we have a list of those months here!\n{list(date_generator(FIRST_MONTH, LAST_MONTH))}')


The month_range() function gets us that the first month is 201501,the last month is 201712, and the total number of months is 36
Also, we have a list of those months here!
['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712']


With those parts sorted, we're now going to add another set of `key-value` pairs to the `ogora_dict` data. We're going to add a `list` for each of the water, oil, gas, and Production days, where each element in the list corresponds to what would be a *column* in the Excel spreadsheet.

We're also going to put in an `if-else` statement to handle the fact that, on some months, some WellZones did not report any Production (or lack thereof) data. For those specific dates we're just gonna put in a `'0'`. 

In [151]:
def list_maker(data: dict):
    values = list()
    for month in date_generator(FIRST_MONTH, LAST_MONTH):
        if month in data.keys():
            values.append(data[month])
        
        else:
            values.append('0')
    
    return values

for wz in ogora_dict.keys():
    for info in [u'DailyOilList', u'DailyGasList', u'DailyWaterList', u'ProductionDaysList']:
        ogora_dict[wz][info] = list_maker(ogora_dict[wz][info[:-4]])

Now that we've done that, let's take a quick peek at that `sample_wz` to see what kind of data we have.

In [152]:
pprint(ogora_dict[sample_wz])

{'API': '608054001101',
 'BlockNum': 'AC   25',
 'DailyGas': {'201501': '000039611',
              '201502': '000031359',
              '201503': '000035747',
              '201504': '000031226',
              '201505': '000028646',
              '201506': '000024157',
              '201507': '000037648',
              '201508': '000021362',
              '201509': '000015559',
              '201510': '000027842',
              '201511': '000028931',
              '201512': '000026885',
              '201601': '000021715',
              '201602': '000026251',
              '201603': '000022491',
              '201604': '000010367',
              '201605': '000025213',
              '201606': '000019664',
              '201607': '000020054',
              '201608': '000019580',
              '201609': '000019719',
              '201610': '000019231',
              '201611': '000018248',
              '201612': '000016021',
              '201701': '000019858',
              '201702': '00

Alright! So we've now got... Pretty much everything we need to make the spreadsheet you wanted!

### Making a CSV

We're going to import the `csv` library, to make a CSV though, later on, we'll also try to use `openpyxl` to make a `.xlsx` file so that we can have the multiple sheets within the file. But for now, let's just make a `.csv` of the Daily Oil Production.

In [157]:
import csv

REQUESTED_DATA = [u'DailyOilList', u'DailyGasList', u'DailyWaterList', u'ProductionDaysList']

data_collector = list()

The first row of the spreadsheet has all of the column headers. So we'll add those to the `data_collector` list, along with all of the months we have the data for.

Then, we'll go into each WellZone, and get the `REQUESTED_DATA`, along with the WellZone specifics, and append those to the `data_collector` list.

In [158]:
data_collector.append([u'Field', u'BlockNum', u'API', u'Int', u'WellZone'] + list(date_generator(FIRST_MONTH, LAST_MONTH)))

for wz in ogora_dict.keys():
    Field = ogora_dict[wz][u'FieldName']
    BlockNum = ogora_dict[wz][u'BlockNum']
    API = ogora_dict[wz][u'API']
    Int = ogora_dict[wz][u'Int']
    
    WellZone = wz
    
    numbers = ogora_dict[wz][REQUESTED_DATA[0]]
    
    data = [Field, BlockNum, API, Int, WellZone] + numbers[:]
    
    data_collector.append(data)

All the data we needed is now collected in the `data_collector` variable. Let's save it as the csv file, and confirm the files look like we want!

In [159]:
with open('data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(data_collector)

The output for that `sample_wz` row that we were working with, in the saved csv file, will look like this:

| Field  | BlockNum | API         | Int | WellZone        | 201501 | 201502 | 201503 | 201504 | 201505 | 201506 | 201507 | 201508 | 201509 | 201510 | 201511 | 201512 | 201601 | 201602 | 201603 | 201604 | 201605 | 201606 | 201607 | 201608 | 201609 | 201610 | 201611 | 201612 |
|--------|----------|-------------|-----|-----------------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| …      | …        | …           | …   | …               | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      |
| Hoover | AC   25  | 6.08054E+11 | S01 | 608054001101S01 | 27289  | 25069  | 27285  | 30248  | 30849  | 29483  | 32401  | 29564  | 29213  | 24072  | 20645  | 20764  | 16225  | 19730  | 19876  | 8433   | 20886  | 19381  | 19899  | 17376  | 16521  | 15677  | 15219  | 15393  |
| …      | …        | …           | …   | …               | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      | …      |

Pretty nifty, huh! Finally, if you need to go in and change some of the details, it should be largely just a matter of formatting the numbers. For example, maybe you don't want the dates shown as `YYYYMM`, but rather `MM/YYYY`. All you have to do is change the way the `date_generator` function yields those dates.

Let's go ahead and use the remainder of the `REQUESTED_DATA` list, and make a CSV for each of the "types" of data.

In [162]:
for request in REQUESTED_DATA:
    data_collector = list()
    
    data_collector.append([u'Field', u'BlockNum', u'API', u'Int', u'WellZone'] + list(date_generator(FIRST_MONTH, LAST_MONTH)))

    for wz in ogora_dict.keys():
        Field = ogora_dict[wz][u'FieldName']
        BlockNum = ogora_dict[wz][u'BlockNum']
        API = ogora_dict[wz][u'API']
        Int = ogora_dict[wz][u'Int']

        WellZone = wz

        numbers = ogora_dict[wz][REQUESTED_DATA[0]]

        data = [Field, BlockNum, API, Int, WellZone] + numbers[:]

        data_collector.append(data)
    
    with open(f'{request}.csv', u'w', newline = u'') as f:
        writer = csv.writer(f)
        writer.writerows(data_collector)

### Summary

So in the above code, and I'll put a summary of all the Python parts in one area here in the appendix, so that it's easier to access/read and copy/paste, we have a way to go through all of the data in the Government's BSEE files, and fill them into your spreadsheet. I'll also get started on getting you an explanation for the same thing, but putting it into an Excel `.xlsx` file, as soon as I learn how to actually use `openpyxl`.

### Appendices/Other Stuff

This section is really only here to make sure that we can validate from where we got our information, if we need to do so again in the future.

- Data Center > Other > Deepwater Natural Gas and Oil Qualified Fields

    - Associates a block num to a field name
    
      - Link: [https://www.data.bsee.gov/Other/Files/DeepQualRawData.zip](https://www.data.bsee.gov/Other/Files/DeepQualRawData.zip)
    
- Data Center > Production > OGOR-A

  - Link: [https://www.data.bsee.gov/Production/Files/ogoradelimit.zip](https://www.data.bsee.gov/Production/Files/ogoradelimit.zip)
    
  - Note that all of the data is stored in the OGOR-A directory, with decrementing years to signify all previous year's data. i.e.,
      - https://www.data.bsee.gov/Production/Files/ogora2018delimit.zip
      - https://www.data.bsee.gov/Production/Files/ogora2017delimit.zip
      - https://www.data.bsee.gov/Production/Files/ogora2016delimit.zip
      
### Appendices/Python Code